# Introduction to Cross Validation

https://scikit-learn.org/stable/modules/cross_validation.html

## Imports


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
pd.set_option('display.max_columns', None)

## Data Example


In [4]:
df = pd.read_csv('../data/Advertising.csv')

In [5]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


----
----
----
## Train | Test Split Procedure 

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Test Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Test Data (by creating predictions and comparing to Y_test)
7. Adjust Parameters as Necessary and repeat steps 5 and 6

In [105]:
# CREATE X and y
X = df.drop('sales', axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Create Model**

In [106]:
from sklearn.linear_model import Ridge

In [107]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha=100)

In [108]:
model.fit(X_train, y_train)

Ridge(alpha=100)

In [109]:
y_pred = model.predict(X_test)

**Evaluation**

In [110]:
from sklearn.metrics import mean_squared_error

In [111]:
mean_squared_error(y_test, y_pred)

7.341775789034129

**Adjust Parameters and Re-evaluate**

In [112]:
model = Ridge(alpha=1)

In [113]:
model.fit(X_train, y_train)

Ridge(alpha=1)

In [114]:
y_pred = model.predict(X_test)

**Another Evaluation**

In [115]:
mean_squared_error(y_test, y_pred)

2.3190215794287514

----
----
----
## Train | Validation | Test Split Procedure 

This is often also called a "hold-out" set, since you should not adjust parameters based on the final test set, but instead use it *only* for reporting final expected performance.

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Validation/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Eval Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Evaluation Data (by creating predictions and comparing to Y_eval)
7. Adjust Parameters as Necessary and repeat steps 5 and 6
8. Get final metrics on Test set (not allowed to go back and adjust after this!)

In [116]:
## CREATE X and y
X = df.drop('sales', axis=1)
y = df['sales']

In [118]:
######################################################################
#### SPLIT TWICE! Here we create TRAIN | VALIDATION | TEST  #########
####################################################################
from sklearn.model_selection import train_test_split

# 70% of data is training data, set aside other 30%
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=101)

# Remaining 30% is split into evaluation and test setsa
# Each is 15% of the original data size
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

In [119]:
# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

**Create Model**

In [120]:
model = Ridge(alpha=100)

In [121]:
model.fit(X_train, y_train)

Ridge(alpha=100)

In [122]:
y_eval_pred = model.predict(X_eval)

**Evaluation**

In [123]:
mean_squared_error(y_eval, y_eval_pred)

7.320101458823872

**Adjust Parameters and Re-evaluate**

In [124]:
model = Ridge(alpha=1)

In [125]:
model.fit(X_train, y_train)

Ridge(alpha=1)

In [126]:
y_eval_pred = model.predict(X_eval)

**Another Evaluation**

In [127]:
mean_squared_error(y_eval, y_eval_pred)

2.3837830750569866

**Final Evaluation (Can no longer edit parameters after this!)**

In [128]:
y_final_test_pred = model.predict(X_test)

In [130]:
mean_squared_error(y_test, y_final_test_pred)

2.254260083800517

----
----
----
## Cross Validation with cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [131]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [132]:
model = Ridge(alpha=100)

In [133]:
from sklearn.model_selection import cross_val_score

In [134]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [135]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [136]:
abs(scores.mean())

8.215396464543607

**Adjust model based on metrics**

In [137]:
model = Ridge(alpha=1)

In [138]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [139]:
abs(scores.mean())

3.344839296530695

**Final Evaluation (Can no longer edit parameters after this!)**

In [140]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [141]:
y_final_test_pred = model.predict(X_test)

In [142]:
mean_squared_error(y_test, y_final_test_pred)

2.3190215794287514

----
----
----

# Cross Validation with cross_validate

The cross_validate function differs from cross_val_score in two ways:

It allows specifying multiple metrics for evaluation.

It returns a dict containing fit-times, score-times (and optionally training scores as well as fitted estimators) in addition to the test score.

For single metric evaluation, where the scoring parameter is a string, callable or None, the keys will be:
        
        - ['test_score', 'fit_time', 'score_time']

And for multiple metric evaluation, the return value is a dict with the following keys:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score is set to False by default to save computation time. To evaluate the scores on the training set as well you need to be set to True.

In [143]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [145]:
model = Ridge(alpha=100)

In [146]:
from sklearn.model_selection import cross_validate

In [147]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [148]:
scores

{'fit_time': array([0.007797  , 0.00246119, 0.00154114, 0.00116777, 0.00137544]),
 'score_time': array([0.00387907, 0.002424  , 0.00185227, 0.00139594, 0.0016613 ]),
 'test_neg_mean_absolute_error': array([-2.31243044, -1.74653361, -2.56211701, -2.01873159, -2.27951906]),
 'test_neg_mean_squared_error': array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
         -8.38562723]),
 'test_max_error': array([ -6.44988486,  -5.58926073, -10.33914027,  -6.61950405,
         -7.75578515])}

In [149]:
scores = pd.DataFrame(scores)

In [150]:
scores

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.007797,0.003879,-2.312430,-9.325530,-6.449885
1,0.002461,0.002424,-1.746534,-4.944962,-5.589261
2,0.001541,0.001852,-2.562117,-11.396652,-10.339140
3,0.001168,0.001396,-2.018732,-7.024211,-6.619504
4,0.001375,0.001661,-2.279519,-8.385627,-7.755785


In [151]:
scores.mean()

fit_time                        0.002869
score_time                      0.002243
test_neg_mean_absolute_error   -2.183866
test_neg_mean_squared_error    -8.215396
test_max_error                 -7.350715
dtype: float64

**Adjust model based on metrics**

In [152]:
model = Ridge(alpha=1)

In [153]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [154]:
scores = pd.DataFrame(scores)

In [155]:
scores

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.002119,0.001841,-1.547117,-3.155132,-3.088300
1,0.001706,0.001961,-1.026044,-1.580870,-2.817441
2,0.001872,0.002270,-1.400793,-5.404556,-9.353209
3,0.002521,0.002503,-1.154251,-2.216545,-4.055856
4,0.001817,0.002026,-1.470222,-4.367094,-6.490922


In [156]:
scores.mean()

fit_time                        0.002007
score_time                      0.002120
test_neg_mean_absolute_error   -1.319685
test_neg_mean_squared_error    -3.344839
test_max_error                 -5.161145
dtype: float64

**Final Evaluation (Can no longer edit parameters after this!)**

In [157]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [158]:
y_final_test_pred  = model.predict(X_test)

In [159]:
mean_squared_error(y_test, y_final_test_pred)

2.3190215794287514

---